In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
import sys
import copy
from copy import deepcopy
import math
from os.path import join as oj
import operator
sys.path.insert(1, oj(sys.path[0], 'sentiment'))

from sentiment import sent_util
import tiling_text
import scores as score_funcs
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torchtext import data, datasets
import agglomerate_text
import visualize_text

from skimage import measure

In [3]:
dset = 'dev'
method = 'cd'

# Not sure what these mean, agglomeration parameters
subtract = True
percentile_include = 90
num_iters = 25
absolute = True
sweep_dim = 1

## Misc functions

In [4]:
# get model, generate bad model
# snapshot_dir = '/accounts/grad/jmurdoch/pytorch/agglomerative_attention/sentiment/results/'
# snapshot_file = oj(snapshot_dir, 'best_snapshot_devacc_85.77981651376147_devloss_0.4260834753513336__iter_3000_model.pt')
snapshot_file = '/scratch/users/vision/chandan/data/saliency/train_subtrees/best_snapshot_devacc_85.77981651376147_devloss_0.4260834753513336__iter_3000_model.pt'
model = sent_util.get_model(snapshot_file)
model.eval()

# get data
inputs, answers, train_iterator, dev_iterator = sent_util.get_sst()

loading /scratch/users/vision/chandan/data/saliency/train_subtrees/best_snapshot_devacc_85.77981651376147_devloss_0.4260834753513336__iter_3000_model.pt


/accounts/projects/vision/.local/lib/python3.5/site-packages/torch/serialization.py:316: SourceChangeWarning: source code of class 'model.LSTMSentiment' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/accounts/projects/vision/.local/lib/python3.5/site-packages/torch/serialization.py:316: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/accounts/projects/vision/.local/lib/python3.5/site-packages/torch/serialization.py:316: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by a

loaded onto gpu...


# find best, worst examples

In [12]:
preds = []
print(len(dev_iterator))
for i, batch in enumerate(dev_iterator):
    lab = batch.label.data[0]
    pred = model(batch)
    preds.append(pred.data[0, lab])

872


sentiment/model.py:30: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
  lstm_out, self.hidden = self.lstm(vecs, self.hidden)


In [13]:
top_idxs = np.argsort(preds)[::-1]
preds[top_idxs[0]]
top_10 = top_idxs[:10]
bot_10 = top_idxs[-10:][::-1]
print('top idxs', top_10)
print('bot idxs', bot_10)

top idxs [ 89  66 315 870  76   2  68  88  11  98]
bot idxs [783 838  19 813 650 235 520 724 286 522]


In [10]:
# Generate bad model
np.random.seed(17042018)
modify_prop = 0.25
bad_model = copy.deepcopy(model)
for param in bad_model.parameters():
    num_params = np.prod(list(param.data.size()))
    num_modify = modify_prop * num_params
    to_modify = np.random.choice([True, False], tuple(param.data.size()), 
                                 p=(modify_prop, 1-modify_prop))
    cpu_param = param.data.cpu().numpy()
    #permuted_vars = cpu_param[to_modify]
    cpu_param[to_modify] = np.random.permutation(cpu_param[to_modify])
    param.data = torch.cuda.FloatTensor(cpu_param) 

In [5]:
# Compute accuracy of bad model

dev_iterator.init_epoch()
model.eval()
num_correct, num_total = 0, 0
for batch_idx, batch in enumerate(dev_iterator):
    pred = bad_model(batch)
    _, max_ind = pred[0].max(0)
    if max_ind.data[0] == batch.label.data[0]:
        num_correct += 1
    num_total += 1
print("Dev accuracy of bad model:", num_correct / num_total)

sentiment/model.py:30: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  lstm_out, self.hidden = self.lstm(vecs, self.hidden)


Dev accuracy of bad model: 0.7981651376146789


In [8]:
batch.label

Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

In [21]:
good_correct = {}
bad_correct = {}
for i, batch in enumerate(dev_iterator):
    lab_num = batch.label.data[0]
    good_max, good_pred = F.softmax(model(batch)).max(1)
    good_pred = good_pred.data[0]; good_max = good_max.data[0]
    bad_max, bad_pred = F.softmax(bad_model(batch)).max(1)
    bad_pred = bad_pred.data[0]; bad_max = bad_max.data[0]
    
    if (good_pred == lab_num or bad_pred == lab_num) and good_pred != bad_pred and batch.text.size(0) > 6 and batch.text.size(0) < 20:
        if good_pred == lab_num:
            good_correct[i] = good_max - F.softmax(bad_model(batch)).data[0, good_pred]
        else:
            bad_correct[i] = bad_max - F.softmax(model(batch)).data[0, bad_pred]


sentiment/model.py:30: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  lstm_out, self.hidden = self.lstm(vecs, self.hidden)
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_la

In [22]:
sorted_good = sorted(good_correct.items(), key=operator.itemgetter(1))[::-1]
sorted_bad = sorted(bad_correct.items(), key=operator.itemgetter(1))[::-1]
good_exs = [a[0] for a in sorted_good[:6]]
bad_exs = [a[0] for a in sorted_bad[:3]]
print(sorted_good[:6])
print(sorted_bad[:3])

[(173, 0.7540188953280449), (274, 0.7136497497558594), (381, 0.7008907198905945), (266, 0.6861542165279388), (378, 0.683125913143158), (168, 0.6483375132083893)]
[(231, 0.782391894608736), (374, 0.600233793258667), (432, 0.5715388506650925)]


In [ ]:
# Compute interaction scores for good, bad models
good_scores = np.zeros(len(dev_iterator))
good_joins = np.zeros(len(dev_iterator))
bad_scores = np.zeros(len(dev_iterator))
bad_joins = np.zeros(len(dev_iterator))

dev_iterator.init_epoch()
for batch_idx, batch in enumerate(dev_iterator):
    lists = visualize_text.agglomerate_and_save(inputs, method, batch, subtract, percentile_include, num_iters, 
                      sweep_dim, model, dset, absolute, fig_name=str(batch_idx), use_bad=False, show_pics=False)
    good_scores[batch_idx], good_joins[batch_idx] = agglomerate_text.interaction_sum(lists)
    lists = visualize_text.agglomerate_and_save(inputs, method, batch, subtract, percentile_include, num_iters, 
                      sweep_dim, bad_model, dset, absolute, fig_name=str(batch_idx), use_bad=False, show_pics=False)
    bad_scores[batch_idx], bad_joins[batch_idx] = agglomerate_text.interaction_sum(lists)
    if batch_idx % 20 == 0:
        print(batch_idx)

In [ ]:
# Save scores
np.save('data/text_good_scores.npy', good_scores)
np.save('data/text_bad_scores.npy', bad_scores)
np.save('data/text_good_joins.npy', good_joins)
np.save('data/text_bad_joins.npy', bad_joins)

In [ ]:
# Load scores
good_scores = np.load('data/text_good_scores.npy')
bad_scores = np.load('data/text_bad_scores0.1.npy')
good_joins = np.load('data/text_good_joins.npy')
bad_joins = np.load('data/text_bad_joins0.1.npy')

In [ ]:
# Normalize scores by number of joins

good_scores /= good_joins
bad_scores /= bad_joins

In [ ]:
good_logits[0]

In [ ]:
# Choose examples
batches = sent_util.get_batches(range(len(dev_iterator) + 1), train_iterator, dev_iterator, dset=dset) # has batch.text, batch.label


# Highest interactions on good
good_exs = set()
bad_exs = set()
for ind in np.argsort(np.abs(good_scores - bad_scores))[::-1]:
    good_logits = model(batches[ind]).data[0]
    bad_logits = bad_model(batches[ind]).data[0]
    if (abs(good_logits[1] - good_logits[0]) > 0.5 and abs(bad_logits[1] - bad_logits[0]) > 0.5 and 
        model.predict(batches[ind]) != bad_model.predict(batches[ind]) and batches[ind].text.size(0) > 6 and
        batches[ind].text.size(0) < 20):
        if model.predict(batches[ind]) == batches[ind].label.data[0]:
            if len(good_exs) < 5:
                good_exs |= set([ind])
        else:
            if len(bad_exs) < 5:
                bad_exs |= set([ind])
        print(good_scores[ind], bad_scores[ind])
        if len(good_exs) == 5 and len(bad_exs) == 5:
            break


In [21]:
def agglomerate_and_save(method, batch, subtract, percentile_include, num_iters, 
                         sweep_dim, model, dset, absolute, fig_name=None, use_bad=False, 
                         wrong_preds=True, mturk=False, no_text=False):
    # output folder
    out_dir = oj('out')
#     folder_name = 'class_diff' if subtract else 'max_class'
#     out_dir = oj('out', method)
#     if use_bad:
#         out_dir = oj(out_dir, 'inaccurate')
#     if wrong_preds:
#         out_dir = oj(out_dir, 'wrong')
#     if not os.path.exists(out_dir):
#         os.makedirs(out_dir)

    # get batch, text, label
    text_orig = [inputs.vocab.itos[int(ind)] for ind in batch.text.data]
    label_actual = int(batch.label) # 0 if positive 1 if negative
    scores_all = model(batch).data.cpu().numpy()[0]
    label_pred = np.argmax(scores_all)
    print('ex_num', fig_name, 'scores_all', scores_all, label_pred, label_actual)
    
    # sweep agglomerative
    lists = agglomerate_text.sweep_agglomerative(model, batch, percentile_include, method, sweep_dim, # only works for sweep_dim = 1
                        text_orig, label_pred, num_iters=num_iters, subtract=subtract, absolute=absolute)

    # visualize
    # visualize_text.visualize_scores(lists['scores_list'][0], label_pred, text_orig, 
    #                             lists['score_orig'], sweep_dim=1, method=method)
    # visualize_text.print_scores(lists, text_orig, num_iters)
    lists = agglomerate_text.collapse_tree(lists)

    
    visualize_text.word_heatmap(text_orig, lists, label_pred, label_actual, method, subtract, mturk, no_text)
    plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight')
    
    viz_args = (text_orig, lists, label_pred, label_actual, method, subtract)
    return viz_args


# generate final viz examples

In [17]:
good_exs = [173, 274, 381, 266, 378, 168]
bad_exs = [231, 374, 432]
if not os.path.exists('out'):
    os.makedirs('out')
for ex in good_exs + bad_exs:
    for i, m in enumerate([model, bad_model]):
        model_name = ['model=good', 'model=bad'][i]
    
        ex_fold = 'ex=good' if ex in good_exs else 'ex=bad'
        batches = sent_util.get_batches(range(len(dev_iterator) + 1), train_iterator, dev_iterator, dset=dset) # has batch.text, batch.label
        batch = batches[ex]


        # ACD
        fig_name = str(ex) + model_name + '_method=acd_' + ex_fold
        viz_args = agglomerate_and_save(method, batch, subtract, percentile_include, num_iters, 
                          sweep_dim, m, dset, absolute, fig_name=fig_name, use_bad=False, mturk=True)

        # CD
        fig_name = str(ex) + model_name + '_method=cd_'+ ex_fold
        (text_orig, lists, label_pred, label_actual, method, subtract) = viz_args
        lists['comps_list'] = [lists['comps_list'][0]]
        visualize_text.word_heatmap(text_orig, lists, label_pred, label_actual, method, subtract, mturk=True)
        plt.savefig(oj('out', fig_name + '.png'), bbox_inches='tight')

        # integrated gradients
        fig_name = str(ex) + model_name + '_method=ig_'+ ex_fold
        scores_ig = score_funcs.ig_scores(batch, model, inputs).reshape((1, -1))
        visualize_text.word_heatmap(text_orig, lists, label_pred, label_actual, method, subtract, mturk=True, data=scores_ig)
        plt.savefig(oj('out', fig_name + '.png'), bbox_inches='tight')
        
        # break_down
        fig_name = str(ex) + model_name + '_method=break_down_'+ ex_fold
        text_orig = [inputs.vocab.itos[int(ind)] for ind in batch.text.data]
        label_actual = int(batch.label) # 0 if positive 1 if negative
        scores_all = model(batch).data.cpu().numpy()[0]
        label_pred = np.argmax(scores_all)
        lists = agglomerate_text.sweep_agglomerative(m, batch, percentile_include, 'break_down', sweep_dim,
                            text_orig, label_pred, num_iters=0, subtract=subtract, absolute=absolute)
        visualize_text.word_heatmap(text_orig, lists, label_pred, label_actual, method, subtract, mturk=True)
        plt.savefig(oj('out', fig_name + '.png'), bbox_inches='tight')



SyntaxError: invalid syntax (<ipython-input-17-93c27129780b>, line 41)

# generate supp viz examples

In [ ]:
# use top_10
# use bot_10
if not os.path.exists('out'):
    os.makedirs('out')

batches = sent_util.get_batches(range(len(dev_iterator) + 1), train_iterator, dev_iterator, dset='dev') # has batch.text, batch.label
print(top_10, bot_10)
for ex in np.concatenate((top_10, bot_10)):
    ex_fold = 'ex=best' if ex in top_10 else 'ex=worst'
    batch = batches[ex]

    # ACD
    fig_name = str(ex) + '_method=acd_' + ex_fold
    viz_args = agglomerate_and_save(method, batch, subtract, percentile_include, num_iters, 
                      sweep_dim, model, dset, absolute, fig_name=fig_name, use_bad=False, mturk=True)